In [1]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import numpy as np
from sklearn.cluster import HDBSCAN
import random
from tqdm import tqdm
# from lib.dataloader import CustomTestDataset
from boilerplate.dataloader import Custom3DTestDataset
import tifffile as tiff
import os
from boilerplate import boilerplate
from scipy.spatial.distance import cdist
from torch.nn.functional import unfold
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

dist_metric = ['cosine']

num_clusters = 4
patch_size = (8,64,64)
centre_size = 5
n_channel = 32
hierarchy_level = 3

In [3]:
data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"

One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_images = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

Test image loaded from path:
/group/jug/Sheida/pancreatic beta cells/download/high_c4/high_c4_source.tif


In [4]:
def count_label_pixels(ground_truth):
    # Get unique labels and their counts
    unique_labels, counts = np.unique(ground_truth, return_counts=True)
    
    # Create a dictionary to store label-wise pixel counts
    label_pixel_counts = dict(zip(unique_labels, counts))
    
    return label_pixel_counts

In [5]:
label_pixel_counts = count_label_pixels(test_ground_truth_image)
print(f"Label-wise pixel counts:{label_pixel_counts}")

Label-wise pixel counts:{-1: 409001638, 0: 107710351, 1: 25910981, 2: 36936035, 3: 23104175}


In [6]:
def select_random_pixels_within_range(ground_truth, labels, pixels_per_class, x_range, y_range, z_range):
    selected_pixels = {}

    for label in labels:
        # Get the coordinates where ground truth equals the label
        coords = np.argwhere(ground_truth == label)
        
        # Filter coordinates based on the specified range
        x_in_range = (coords[:, 0] >= x_range[0]) & (coords[:, 0] <= x_range[1])
        y_in_range = (coords[:, 1] >= y_range[0]) & (coords[:, 1] <= y_range[1])
        z_in_range = (coords[:, 2] >= z_range[0]) & (coords[:, 2] <= z_range[1])
        
        # Keep only the coordinates that satisfy all range conditions
        filtered_coords = coords[x_in_range & y_in_range & z_in_range]
        
        # Check if there are enough pixels for the current label after filtering
        if len(filtered_coords) < pixels_per_class:
            raise ValueError(f"Not enough pixels for label {label} within the specified range. Found {len(filtered_coords)}, required {pixels_per_class}")
        
        # Randomly select pixels_per_class coordinates from filtered_coords
        chosen_indices = np.random.choice(len(filtered_coords), pixels_per_class, replace=False)
        chosen_coords = filtered_coords[chosen_indices]
        
        # Store selected coordinates for the label
        selected_pixels[label] = chosen_coords

    return selected_pixels


In [7]:
labels_of_interest = [0, 1, 2, 3]
z , y, x = test_images.shape
x_range = (64, x - 64)
y_range = (64, y - 64)
z_range = (64, z - 64)


In [24]:
selected_pixel_coords = select_random_pixels_within_range(test_ground_truth_image, labels_of_interest, pixels_per_class=256, x_range=x_range, y_range=y_range, z_range=z_range)

In [25]:
for label, coords in selected_pixel_coords.items():
    print(f"Label {label}: {coords}")

Label 0: [[ 580   82  605]
 [ 646  414  343]
 [ 437  162  610]
 [ 556  262  230]
 [ 973  323  558]
 [ 578  219  219]
 [ 683  478  619]
 [ 784  440  523]
 [ 889  395  328]
 [ 393  456  464]
 [ 675  415  696]
 [ 839  337  367]
 [ 691  155  662]
 [ 212  293  559]
 [ 546  390  263]
 [ 428  180  681]
 [ 490  252  800]
 [ 299  271  883]
 [ 367  201  463]
 [ 157  313  760]
 [ 294  131  505]
 [ 666  201  196]
 [ 593  376  827]
 [ 215  392  561]
 [ 591  160  396]
 [ 583  179  595]
 [ 867  300  898]
 [ 623  310  258]
 [ 368  211  937]
 [ 829  450  412]
 [ 780  353  523]
 [ 106  306  882]
 [ 722  300  621]
 [ 788  367  521]
 [ 572  468  392]
 [ 421  102  540]
 [ 758  293  109]
 [ 749  481  556]
 [ 425  151  914]
 [ 735  295  587]
 [1004  382  661]
 [ 877  287  655]
 [ 308  363  454]
 [ 310  210  947]
 [ 711  374  906]
 [ 431  237  344]
 [ 876  354  174]
 [ 709  418  202]
 [ 468  416  415]
 [ 670  387  396]
 [ 506  190  305]
 [ 621  211  873]
 [  70  334  834]
 [ 911  269  546]
 [ 684  289  144]
 

In [26]:
import pickle

def save_as_pickle(selected_pixel_coords, filename):
    with open(filename, 'wb') as f:
        pickle.dump(selected_pixel_coords, f)

# Usage
save_as_pickle(selected_pixel_coords, 'selected_pixel_coords.pkl')


In [27]:
def extract_patches(image, coords, patch_size=64):
    half_size = patch_size // 2
    patches = []

    # Get the shape of the image
    z_max, y_max, x_max = image.shape

    for coord in coords:
        z, y, x = coord

        # Calculate the boundaries of the patch
        x_start = max(0, x - half_size)
        x_end = min(x_max, x + half_size)
        y_start = max(0, y - half_size)
        y_end = min(y_max, y + half_size)
        z_start = max(0, z - 4)
        z_end = min(z_max, z + 4)
        
        # Extract the patch from the image
        patch = image[z_start:z_end, y_start:y_end, x_start:x_end]
        
        # Check if the patch is the required size, otherwise pad with zeros
        if patch.shape != (8, patch_size, patch_size):
            print("something is wrong")

        
        # Store the patch
        patches.append(patch)

    return np.array(patches)

In [32]:
all_patches_tensors = []

for label, coords in selected_pixel_coords.items():
    patches = extract_patches(test_images, coords, patch_size=64)
    patches_tensor = torch.tensor(patches, dtype=torch.float32).unsqueeze(1)
    print(f"Patches for label {label}: {patches_tensor.shape}")  # The shape should be (256, 8, 64, 64)
    all_patches_tensors.append(patches_tensor)

combined_tensor = torch.stack(all_patches_tensors, dim=0)
print(combined_tensor.shape)

Patches for label 0: torch.Size([256, 1, 8, 64, 64])
Patches for label 1: torch.Size([256, 1, 8, 64, 64])
Patches for label 2: torch.Size([256, 1, 8, 64, 64])
Patches for label 3: torch.Size([256, 1, 8, 64, 64])
torch.Size([4, 256, 1, 8, 64, 64])


In [33]:
model_dir = "/group/jug/Sheida/HVAE/3D/"
model_versions = [4,5,6,7,8]
batch_size = 256
for model_v in model_versions:
    model = torch.load(model_dir+'v0'+str(model_v)+"/model/3D_HVAE_best_vae.net")
    data_mean = model.data_mean
    data_std = model.data_std
    model.mode_pred = True
    model.eval()
    device = model.device
    print(f"Processing 1000 pixels with model version {model_v}")
    index = 0 
    all_mus = np.zeros((batch_size, 43008), dtype=np.float16)
    with torch.no_grad():
        for batch in tqdm(combined_tensor):
            batch = batch.to(device)
            batch = (batch - data_mean) / data_std
            output = model(batch)
            mu_test = torch.cat([torch.mean(output["mu"][i],dim=2).reshape(batch_size, -1) for i in range(hierarchy_level)], dim=1)
            mu_test = np.array(mu_test.cpu().numpy())
            all_mus[index:index+batch_size] = mu_test
            index += batch_size

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(all_mus)
    cluster_labels = cluster_labels.reshape(4, 256)
    tiff.imwrite(f"{model_dir}v0{model_v}/1000pixels.tif", cluster_labels)

Processing 1000 pixels with model version 4


  0%|          | 0/4 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  0%|          | 0/4 [00:52<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 